In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle
import pickle
records = []
df2=pd.DataFrame()
with open("\Dataset\\MovieSummaries\\plot_summaries.txt","r",encoding="utf-8") as infile:
 for line in infile:
    parts = re.split(r'\t+(?=\{)|(?<=\})|\t+', line)
    record={}
    record['Wikipedia movie ID'] = parts[0] 
    record['PlotSummary'] = parts[1]
    

    records.append(record)
df2=pd.DataFrame(records)


In [ ]:
df2.columns=["Wikipedia movie ID","PlotSummary"]
df2.head(5)

,Wikipedia movie ID,PlotSummary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [ ]:
records = []
with open("\\Dataset\\MovieSummaries\\movie.metadata.tsv","r",encoding="utf-8") as infile:
 for line in infile:
    parts = re.split(r'\t+(?=\{)|(?<=\})|\t+', line)
    record={}
    record['Wikipedia movie ID'] = parts[0] 
    record['Freebase movie ID'] = parts[1]
    record['Movie name'] = parts[2]
    record['Movie release date'] = parts[3] 

    records.append(record)
df1=pd.DataFrame(records)

In [28]:
df1.head(5)

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16
2,28463795,/m/0crgdbh,Brun bitter,1988
3,9363483,/m/0285_cd,White Of The Eye,1987
4,261236,/m/01mrr1,A Woman in Flames,1983


In [3]:
df.head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,PlotSummary
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,"Set in the second half of the 22nd century, th..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,A series of murders of rich young women throug...
4,261236,/m/01mrr1,A Woman in Flames,1983,"Eva, an upper class housewife, becomes frustra..."


In [ ]:
df.to_csv("\\Dataset\\MovieSummaries\\movie_project.csv",index=False,encoding="utf-8")

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

df=pd.read_csv("\Dataset\\MovieSummaries\\movie_project.csv",encoding="utf-8")

In [ ]:

def tokenizer(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = emoji.demojize(text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove @mentions
    text = re.sub(r'#', '', text) # remove hashtags
    text = re.sub(r'RT[\s]+', '', text) # remove RT
    text = re.sub(r'https?:\/\/\S+', '', text) # remove hyperlinks
    words= text.lower().split()
    words = [word for word in words if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words
    

In [ ]:

tfi=TfidfVectorizer(tokenizer=tokenizer, stop_words='english')
df['PlotSummary'] = df['PlotSummary'].fillna('')
matrix_tf = tfi.fit_transform(df['PlotSummary'])



C:\Users\MANISH\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\MANISH\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  warnings.warn(
C:\Users\MANISH\AppData\Local\Temp\ipykernel_27580\2138906793.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
tfi=pickle.load(open(os.path.join("\\Dataset\\MovieSummaries\\","tfid.pkl"),"rb"))
matrix_tf=pickle.load(open(os.path.join("\Dataset\\MovieSummaries\\","matrix_tf.pkl"),"rb"))

In [ ]:
def get_top_5_movies(prompt):
    prompt_vector = tfi.transform([prompt])
    similarity_scores = cosine_similarity(prompt_vector, matrix_tf).flatten()
    top_indices = similarity_scores.argsort()[-5:][::-1]
    return df.iloc[top_indices][['Wikipedia movie ID', 'Movie name', 'Movie release date','PlotSummary']]
prompt = "A futuristic story set in space with action and adventure."
top_movies = get_top_5_movies(prompt)
top_movies

,Wikipedia movie ID,Movie name,Movie release date,PlotSummary
35776,29281218,Torn,2009-09-01,A man's inability to see how his actions affec...
64095,18394422,002 Operazione Luna,1965,When a Soviet space craft is lost in space wit...
45325,20695434,Star Pink,1978-12-13,Pink runs an intergalactic gas filling station...
84,16686643,One Small Step: The Story of the Space Chimps,1995,"Told through archival photos and footage, spac..."
73126,35889525,The Merry Wives of Windsor,1950-10-22,"In Elizabethan England, Sir John Falstaff is e..."
